In [8]:
import sys
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import joblib

# --- INICIO: Corrección de rutas para importación y archivos ---
# Obtener la ruta del directorio del script (notebooks)
NOTEBOOKS_DIR = os.path.dirname(os.path.abspath(__file__))
# Subir dos niveles para llegar a la raíz del proyecto
PROJECT_ROOT = os.path.abspath(os.path.join(NOTEBOOKS_DIR, os.pardir, os.pardir))
# Añadir la ruta raíz al path de Python para encontrar el módulo 'src'
sys.path.append(PROJECT_ROOT)

# Ahora la importación de funciones personalizadas funcionará
from src.utils.functions import preparar_datos, evaluar_clasificador, graficar_curva_roc, graficar_matriz_confusion
# --- FIN: Corrección de rutas ---


# --- Importaciones de Modelos y Métricas de Scikit-learn ---
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_curve, auc
)

print("Librerías y módulos cargados exitosamente.")
print(f"Raíz del proyecto establecida en: {PROJECT_ROOT}")



In [ ]:
# --- Cargar y Preparar los Datos ---
# Construir la ruta al archivo de datos limpios
ruta_df_limpio = os.path.join(PROJECT_ROOT, 'src', 'data', 'processed', 'df_clean.csv')

# Cargar los datos
df_limpio = pd.read_csv(ruta_df_limpio)
print(f"\nDatos cargados desde: {ruta_df_limpio}")
print("Primeras filas del DataFrame limpio:")
print(df_limpio.head())

# Preparar los datos para el entrenamiento y la prueba usando la función del módulo utils
X_train, y_train, X_test, y_test = preparar_datos(df_limpio)

print("\nDimensiones de los conjuntos de datos:")
print(f"X_train: {X_train.shape}")
print(f"y_train: {y_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"y_test: {y_test.shape}")

Hiperparametros

In [ ]:
# --- Optimización de Hiperparámetros con RandomizedSearchCV ---
# Nota: RandomizedSearchCV es más rápido que GridSearchCV para espacios de búsqueda grandes.

# 1. Regresión Logística
print("\n--- Optimizando Regresión Logística ---")
param_grid_lr = {
    'penalty': ['l1', 'l2'],
    'C': np.logspace(-4, 4, 20),
    'solver': ['liblinear', 'saga'], # Solvers que soportan l1 y l2
    'max_iter': [100, 1000, 2500]
}
lr = LogisticRegression(random_state=42)
grid_lr = RandomizedSearchCV(lr, param_grid_lr, cv=5, scoring="accuracy", n_jobs=-1, n_iter=20, random_state=42)
grid_lr.fit(X_train, y_train)
lr_best = grid_lr.best_estimator_
print(f"Mejores parámetros: {grid_lr.best_params_}")


# 2. Random Forest
print("\n--- Optimizando Random Forest ---")
param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [80, 90, 100, 110],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
rf = RandomForestClassifier(random_state=42)
grid_rf = RandomizedSearchCV(rf, param_grid_rf, cv=5, scoring='accuracy', n_jobs=-1, n_iter=20, random_state=42)
grid_rf.fit(X_train, y_train)
rf_best = grid_rf.best_estimator_
print(f"Mejores parámetros: {grid_rf.best_params_}")


# --- Evaluación de los Modelos Optimizados ---
best_models = [
    ("Logistic Regression", lr_best),
    ("Random Forest", rf_best)
]

results_list = []

for model_name, model in best_models:
    print(f"\n--- Evaluando modelo optimizado: {model_name} ---")
    metrics = evaluar_clasificador(model, X_train, y_train, X_test, y_test)
    results_list.append({"Model": model_name, **metrics})
    
    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred))

# Convertir la lista de resultados a un DataFrame
results_df = pd.DataFrame(results_list)
print("\n--- Resultados Comparativos de Modelos Optimizados ---")
print(results_df)

# Guardar la tabla de resultados para la app
ruta_resultados = os.path.join(PROJECT_ROOT, 'src', 'data', 'processed', 'df_resultados.csv')
results_df.to_csv(ruta_resultados, index=False)
print(f"Resultados guardados en: {ruta_resultados}")


# --- Guardar el Mejor Modelo ---
# Basado en los resultados, elegimos el mejor modelo (ej. Regresión Logística por su balance)
mejor_modelo_final = lr_best

# Crear el directorio si no existe
os.makedirs(os.path.join(PROJECT_ROOT, 'src', 'models'), exist_ok=True)

# Guardar el modelo
ruta_modelo_final = os.path.join(PROJECT_ROOT, 'src', 'models', 'modelo_lr_mejor.pkl')
joblib.dump(mejor_modelo_final, ruta_modelo_final)

print(f"\nMejor modelo (Logistic Regression) guardado en: {ruta_modelo_final}")
